#### Configure Chinook DB from Google
- url : https://storage.googleapis.com/benchmarks-artifacts/chinook/Chinook.db

In [6]:
import requests
url = "https://storage.googleapis.com/benchmarks-artifacts/chinook/Chinook.db"
response = requests.get(url)
if response.status_code == 200 :
    # Open local file in write mode (binary)
    with open("Chinook.db", "wb") as file:
        file.write(response.content)
    print(f"File downloaded and saved as Chinook.db")
else:
    print(f"Error connecting to DB : {response.status_code}")

File downloaded and saved as Chinook.db


In [7]:
# Define SQL Database wrapper available in langchain
from langchain_community.utilities import SQLDatabase
db = SQLDatabase.from_uri("sqlite:///Chinook.db")
print(f"db.dialect : {db.dialect}")
print(f"db.get_usable_tables_names : {db.get_usable_table_names()}")
db.run("SELECT * FROM Artist LIMIT 10;")

db.dialect : sqlite
db.get_usable_tables_names : ['Album', 'Artist', 'Customer', 'Employee', 'Genre', 'Invoice', 'InvoiceLine', 'MediaType', 'Playlist', 'PlaylistTrack', 'Track']


"[(1, 'AC/DC'), (2, 'Accept'), (3, 'Aerosmith'), (4, 'Alanis Morissette'), (5, 'Alice In Chains'), (6, 'Antônio Carlos Jobim'), (7, 'Apocalyptica'), (8, 'Audioslave'), (9, 'BackBeat'), (10, 'Billy Cobham')]"

In [8]:
# Utility Functions, Error handling
# Following code borrowed from - https://langchain-ai.github.io/langgraph/tutorials/sql-agent/#utility-functions
from typing import Any

from langchain_core.messages import ToolMessage
from langchain_core.runnables import RunnableLambda, RunnableWithFallbacks
from langgraph.prebuilt import ToolNode


def create_tool_node_with_fallback(tools: list) -> RunnableWithFallbacks[Any, dict]:
    """
    Create a ToolNode with a fallback to handle errors and surface them to the agent.
    """
    return ToolNode(tools).with_fallbacks(
        [RunnableLambda(handle_tool_error)], exception_key="error"
    )


def handle_tool_error(state) -> dict:
    error = state.get("error")
    tool_calls = state["messages"][-1].tool_calls
    return {
        "messages": [
            ToolMessage(
                content=f"Error: {repr(error)}\n please fix your mistakes.",
                tool_call_id=tc["id"],
            )
            for tc in tool_calls
        ]
    }

In [9]:
# Define LLM
# Basic Setup
import os
from dotenv import load_dotenv, find_dotenv

# Load local env file
_ = load_dotenv(find_dotenv())

from langchain_openai import ChatOpenAI
model = ChatOpenAI(model="gpt-4o-mini", api_key=os.environ['OPENAI_API_KEY'])

In [ ]:
# Define SQL tool for agent
from langchain_community.agent_toolkits import SQLDatabaseToolkit
sql_toolkit = SQLDatabaseToolkit(
    db=db,
    llm=model,
)
tools = sql_toolkit.get_tools()

# Fetch all available tables from database
list_tables = next(tool for tool in tools if tool.name == "sql_db_list_tables")

# Fetch DDL for a table
get_schema = next(tool for tool in tools if tool.name == "sql_db_schema")

print(f"list tables : {list_tables.invoke("")} ")
print(f"list schema : {get_schema.invoke("Artist")}")